In [1]:
import numpy as np
import rasterio
import shapely
from czml3 import Document, Preamble

from czml3_ext import packets
from czml3_ext.colours import RGBA, create_palette
from czml3_ext.helpers import get_border, get_contours

## Border
A single border

In [2]:
border = packets.border("Israel")

Multiple borders

In [3]:
borders = packets.border(["syria", "lebanon", "jordan"])

## Sensors
A single sensor

In [4]:
sensor1 = packets.sensor_polyline(
    np.array([[31.4], [34.7], [0]]), 10, 10, 30, 20, 10_000, 5000
)

Multiple sensors

In [5]:
sensors1 = packets.sensor_polyline(
    np.array([[[31.75], [34.72], [0]], [[31.8], [34.68], [0]]]),
    [50, 90],
    [60, 30],
    [10, 50],
    [20, 20],
    [10_000, 20_000],
    [5_000, 2_000],
)

In [6]:
sensors2 = packets.sensor_polygon(
    np.array([[[31.75], [34.72], [0]], [[31.8], [34.68], [0]]]),
    [50, 90],
    [60, 30],
    [10, 50],
    [20, 20],
    [10_000, 20_000],
    [5_000, 2_000],
)

## Grid
Create a grid in Israel

In [7]:
ddm_LLA_israel = get_border("israel")
poly_shape = shapely.Polygon(ddm_LLA_israel[:, [0, 1], 0].tolist())

In [8]:
x = np.linspace(ddm_LLA_israel[:, 1, 0].min(), ddm_LLA_israel[:, 1, 0].max(), 20)
y = np.linspace(ddm_LLA_israel[:, 0, 0].min(), ddm_LLA_israel[:, 0, 0].max(), 20)
xv, yv = np.meshgrid(x, y)
ddm_points = []
for i in range(x.shape[0]):
    for j in range(y.shape[0]):
        if not poly_shape.contains(shapely.Point(yv[j, i], xv[j, i])):
            continue
        ddm_points.append([yv[j, i], xv[j, i], 0])
ddm_LLA_points = np.array(ddm_points).reshape((-1, 3, 1))

Define the colour palette

In [9]:
rgba = create_palette([RGBA.blue, RGBA.white, RGBA.blue], ddm_LLA_points.shape[0])

Create the grid

In [10]:
grid = packets.grid(ddm_LLA_points, rgba)

## Coverage

In [11]:
with rasterio.open("data.tif") as src:
    arr = src.read(1)
    origin_x, origin_y = src.transform * (0, 0)
    delta_x = src.transform[0]
    delta_y = src.transform[4]

In [12]:
coverages, holes = get_contours(
    arr == 5,
    origin_x,
    delta_x,
    origin_y,
    delta_y,
)

In [13]:
vs = packets.coverage(coverages, holes)

In [14]:
sensor_coverage = packets.sensor_polyline(
    np.array(
        [
            [origin_y + arr.shape[0] / 2 * delta_y],
            [origin_x + arr.shape[1] / 2 * delta_x],
            [0],
        ]
    ),
    320,
    20,
    180,
    20,
    10_000,
)

## Export
Export to file as you would any other `czml3.Document`.

In [15]:
doc = Document(
    [Preamble(name="simple")]
    + border
    + borders
    + sensor1
    + sensors1
    + sensors2
    + grid
    + vs
    + sensor_coverage
)

In [17]:
with open("example.czml", "w") as f:
    doc.dump(f)